In [ ]:
import cv2
import matplotlib.pyplot as plt
from skimage.color import gray2rgb
import pydicom as dicom
from pydicom import dcmread
import numpy as np
import pandas as pd
from glob import glob
from tqdm.notebook import tqdm
import h5py
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization, Dense, Activation, Flatten, Dropout, Input
from tensorflow.keras.applications import VGG19 as PTM_19, VGG16 as PTM_16
from keras.models import Sequential
from keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam as adm , SGD as sgd
from tensorflow.keras.applications.vgg16 import preprocess_input as input16
from tensorflow.keras.applications.vgg19 import preprocess_input as input19
import matplotlib.patches as patches
from keras.utils.vis_utils import plot_model
import tensorflow as tf

In [ ]:
h5file_path = '../input/coviddetectiondataset/CovidDetectionTrainingData.h5'
classes_file_path = '../input/classes/classes.npy'

In [ ]:
h5Dataset = h5py.File(h5file_path, 'r')
classes = np.load(classes_file_path)

In [ ]:
data_file_names = h5Dataset["data_file_names"][:]
data_images = h5Dataset["data_images"][:]
data_target_cc = h5Dataset["data_target_cc"][:]
data_targets_bbox = h5Dataset["data_targets_bbox"][:]
data_file_names.shape, data_images.shape, data_target_cc.shape, data_targets_bbox.shape

In [ ]:
training_testing_split = train_test_split(data_images, data_target_cc, data_targets_bbox, data_file_names, test_size=0.3, random_state=42)
X_train_images, X_test_images = training_testing_split[:2]
y_train_clf, y_test_clf = training_testing_split[2:4]
y_train_bbox, y_test_bbox = training_testing_split[4:6]
fn_train, fn_test = training_testing_split[6:]

In [ ]:
X_train_images.shape, X_test_images.shape, y_train_clf.shape, y_test_clf.shape, y_train_bbox.shape, y_test_bbox.shape, fn_train.shape, fn_test.shape

In [ ]:
del data_images, data_target_cc, data_targets_bbox, data_file_names

In [ ]:
mtf = tf.keras.callbacks

In [ ]:
GPU_DEVICE="/device:GPU:0"

In [ ]:
ptm_19_weights_path = '../input/vgg19weights1/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
ptm_19 = PTM_19(weights=ptm_19_weights_path,
            include_top=False, pooling='avg')
ptm_19.trainable = False

In [ ]:
ptm_19.summary()

In [ ]:
training_features_19 = []
print("Extracting Training Features ...")
for train_image_19 in tqdm(X_train_images):
  with tf.device(GPU_DEVICE):
    img_19 = np.expand_dims(train_image_19, axis=0)
    training_feature_19 = ptm_19.predict(input19(img_19))[0]
    training_features_19.append(training_feature_19)
training_features_19 = np.array(training_features_19)
print("Done Extracting Training Features.")

In [ ]:
training_features_19.shape

In [ ]:
testing_features_19 = []
print("Extracting Testing Features ...")
for test_image_19 in tqdm(X_test_images):
  with tf.device(GPU_DEVICE):
    tst_img_19 = np.expand_dims(test_image_19, axis=0)
    testing_feature_19 = ptm_19.predict(input19(tst_img_19))[0]
    testing_features_19.append(testing_feature_19)
testing_features_19 = np.array(testing_features_19)
print("Done Extracting Testing Features.")

In [ ]:
testing_features_19.shape

In [ ]:
model_19_clf = Sequential([
    Dense(1024, input_shape=(512,)),
    Activation('relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(512),
    Activation('relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256),
    Activation('relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256),
    Activation('relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256),
    Activation('relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(4),
    Activation('softmax'),
])

adam_lr_clf = adm(learning_rate =1e-4,  beta_1=0.9, beta_2=0.999)
model_19_clf.compile(optimizer = adam_lr_clf, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model_19_bbox = Sequential([
    Dense(1024, input_shape=(512,)),
    Activation('relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(512),
    Activation('relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256),
    Activation('relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256),
    Activation('relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256),
    Activation('relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(4),
    Activation('sigmoid'),
])

adam_lr_bbox = adm(learning_rate = 0.001, beta_1=0.9, beta_2=0.999, decay=0.001)
model_19_bbox.compile(loss='mean_squared_error', optimizer=adam_lr_bbox, metrics=['mean_absolute_error'])

In [ ]:
epochs = 63
batch_size = 8

In [ ]:
with tf.device(GPU_DEVICE):
  history_clf = model_19_clf.fit(training_features_19, y_train_clf,
                                 validation_data=(testing_features_19, y_test_clf),
                                 batch_size=batch_size,
                                 epochs=epochs, verbose=1, shuffle=True, )#, shuffle=True, callbacks=[model_checkpoint_callback])

In [ ]:
history_clf_19 = history_clf.history

print("VGG_19 CLASSIFICATION ACCURACY GRAPH")
plt.plot(history_clf_19['accuracy'])
plt.plot(history_clf_19['val_accuracy'])
plt.title('Accuracy Of VGG-19')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
print("VGG_19 CLASSIFICATION LOSS GRAPH")
plt.plot(history_clf_19['loss'])
plt.plot(history_clf_19['val_loss'])
plt.title('Loss Of VGG-19')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
evaluation_19 = model_19_clf.evaluate(testing_features_19, y_test_clf)

In [ ]:
with tf.device(GPU_DEVICE):
  history_bbox = model_19_bbox.fit(training_features_19, y_train_bbox, verbose=1, batch_size=batch_size, epochs=epochs, validation_data=(testing_features_19, y_test_bbox))

In [ ]:
history_bbox_19 = history_bbox.history

print("VGG_19 BOUNDING BOX REGRESSION LOSS GRAPH")
plt.plot(history_bbox_19['loss'])
plt.plot(history_bbox_19['val_loss'])
plt.title('Loss Of VGG-19')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
print("VGG_19 BOUNDING BOX REGRESSION MEAN ABSOLUTE ERROR GRAPH")
plt.plot(history_bbox_19['mean_absolute_error'])
plt.plot(history_bbox_19['val_mean_absolute_error'])
plt.title('MAE Of VGG-19')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
index = 135
test_img = testing_features_19[index]
test_img = np.expand_dims(test_img, axis=0)
pred_clf = model_19_clf.predict(test_img)
pred_bbox = model_19_bbox.predict(test_img)

In [ ]:
pred_clf[0], y_test_clf[index], pred_bbox[0], y_test_bbox[index]

In [ ]:
testing_img_path = '../input/siim-covid19-detection/test/'

In [ ]:
pred_images_fp = glob(f'{testing_img_path}*/*/*')

In [ ]:
image_size = (224,224)

In [ ]:
results = []
for pred_img_fp in tqdm(pred_images_fp):
  try:
    with tf.device(GPU_DEVICE):
      sutdy_id = f'{pred_img_fp.split("/")[5]}_study'
      ds = dcmread(pred_img_fp) # reading image from the created path
      image = np.array(ds.pixel_array) # converting image pixels into numpy array
      image = cv2.resize(image, image_size) # resizing image
      image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB) #convert to 3 channel
      image = np.expand_dims(image, axis=0)
      image = ptm_19.predict(input19(image))[0]
      image = np.expand_dims(image, axis=0)
      clf_pred = model_19_clf.predict(image)
      clf_pred_val = clf_pred[0]
      clf_pred_val = np.argmax(clf_pred_val)
      class_pred = classes[clf_pred_val]
      confidence = clf_pred[0][clf_pred_val]
      bbox = model_19_bbox.predict(image)
      x,y,w,h = bbox[0]
      pred_str = None
      if class_pred == classes[0]:
        pred_str = f'{class_pred} {confidence} 0 0 0 0'
      else:
        pred_str = f'{class_pred} {confidence} {x} {y} {w} {h}'
      results.append([sutdy_id, pred_str])
  except:
    pass#print("Exception Occured")

In [ ]:
dataframe = pd.DataFrame(results, columns=['id', 'PredictionString'])
dataframe.head(20)

In [ ]:
dataframe.shape

In [ ]:
dataframe.to_csv("submission.csv", index=False)